In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data # 获取迭代数据
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.autograd import Variable # 获取变量

import torch.optim as optim
import torchvision
from torchvision.datasets import mnist # 获取数据集
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [2]:
transform = transforms.Compose(
    [transforms.ToTensor()])

In [21]:
image = np.load(data_file)[0]

In [31]:
aa = torch.tensor(image)

In [35]:
b = transform(image)

In [38]:
c = b.reshape(1,21,21)

In [44]:
class CustomDataset_attr(Dataset):
    def __init__(self,label_file, data_file, transform = None):
    
        super().__init__()
        self.attr_ = np.load(label_file).tolist()
        self.label_file = label_file
        self.data_file = data_file
        self.transform = transform
    
    def __len__(self):
        #展示数据中总共有多少个样本
        return len(self.attr_)
    
    def __info__(self):
        print("CustomData")
        print("\t Number of samples: {}".format(len(self.attr_)-1))

    
    def __getitem__(self,idx):
        #保证idx不是一个tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        self.attr_ = np.load(self.label_file).tolist()
        #加载tensor
        image = np.load(self.data_file)[idx]
        label = int(self.attr_[idx])
        
        if self.transform != None:
            image = self.transform(image).reshape(1,21,21)
            
        sample = (image,label)
        return sample

In [45]:
#标签的文件
label_file = 'data/label.npy'
data_file = 'data/data.npy' #tensor

data = CustomDataset_attr(label_file,data_file,transform=transform)

In [16]:
a = np.load(data_file)

In [20]:
a[1].shape

(1, 21, 21)

In [46]:
train_data = CustomDataset_attr(label_file,data_file,transform=transform)

In [47]:
#数据包装
batch_size = 4
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [45]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [96]:
ne1 = nn.Sequential(nn.Conv2d(1,32,3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(32,64,3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )

In [48]:
class EagleC_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features_ = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(32,64,3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )

        self.clf_ = nn.Sequential(nn.Dropout(0.5)
                                  ,nn.Linear(64*5*5,512),nn.ReLU(inplace=True)
                            ,nn.Linear(512,6),nn.Sigmoid()
                                 )
    
    def forward(self,x):
        x = self.features_(x) #用特征提取的架构提取特征
        x = x.view(-1,64*5*5) #调整数据结构，拉平数据
        output = self.clf_(x)
        return output

In [49]:
net = EagleC_CNN()

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [51]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


In [53]:
PATH = './model/cifar_net.pth'
torch.save(net.state_dict(), PATH)